In [1]:
import sys
# !{sys.executable} -m pip install sklearn
# !{sys.executable} -m pip install torch
import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import warnings 
from ast import literal_eval
from laplotter import LossAccPlotter
from laplotter_prediction import PredictionPlotter
import math

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
torch.manual_seed(43) # for reproducibility

In [2]:
# Depthless
# training_set = pd.read_csv('../CSV_files/KITTI/SIFT/kitti_SIFT-sensorsALL.csv')
# training_set = pd.read_csv('../CSV_files/KITTI/SIFT/kitti0093_SIFT-sensorsALL.csv')
training_set = pd.read_csv('../CSV_files/KITTI/SIFT/kitti0095_SIFT-sensorsALL.csv')

# training_set = pd.read_csv('../CSV_files/SIFT/kitti0095_SIFTDepthless-sensorsALL.csv')

dataSet_len = len(training_set)
print(f"number of frames: {dataSet_len}")
featuresAreCoordinates = True # When false, displacement is used.
depthStatus = True # Using depth information
testModel = True # Avoid saving the model and re-ploting the loss
sequence_len = 1

number of frames: 267


In [3]:
distance_sensorData = [literal_eval(i) for i in training_set['sensors_e']]
x_sensorData = [literal_eval(i) for i in training_set['sensors_x']]
y_sensorData = [literal_eval(i) for i in training_set['sensors_y']]
z_sensorData = [literal_eval(i) for i in training_set['sensors_d']]
target_dataX = training_set['x_displacement']
target_dataY = training_set['y_displacement']
coordinatesSRC = [literal_eval(i) for i in training_set['coordinates_src']]
coordinatesDST = [literal_eval(i) for i in training_set['coordinates_dst']]
depthSRC = [literal_eval(i) for i in training_set['depth_src']]
depthDST = [literal_eval(i) for i in training_set['depth_dst']]

In [4]:
numberOfSensorsList = [len(i) for i in distance_sensorData]
meanOfSensors = sum(numberOfSensorsList)//len(numberOfSensorsList)
maxNumberSensors = max(numberOfSensorsList)
minNumberSensors = min(numberOfSensorsList)
meanOfSensors = 51
print(f"max amount of sensors: {maxNumberSensors}")
print(f"min amount of sensors: {minNumberSensors}")
print(f"mean amount of sensors: {meanOfSensors}")

max amount of sensors: 69
min amount of sensors: 22
mean amount of sensors: 51


In [5]:
def standarizeSensors(list_sensors, threshold):
    while len(list_sensors) != threshold:
        if len(list_sensors) < threshold:
            list_sensors.append(0.0)
        elif len(list_sensors) > threshold:
            list_sensors = list_sensors[:threshold]
    return list_sensors

In [6]:
# trim to normalized number of sensors
distance_sensorData = [standarizeSensors(i, meanOfSensors) for i in distance_sensorData]
x_sensorData = [standarizeSensors(i, meanOfSensors) for i in x_sensorData]
y_sensorData = [standarizeSensors(i, meanOfSensors) for i in y_sensorData]
z_sensorData = [standarizeSensors(i, meanOfSensors) for i in z_sensorData]
depthSRC = [standarizeSensors(i, meanOfSensors) for i in depthSRC]
depthDST = [standarizeSensors(i, meanOfSensors) for i in depthDST]
coordinatesSRC = [standarizeSensors(i, meanOfSensors*2) for i in coordinatesSRC]
coordinatesDST = [standarizeSensors(i, meanOfSensors*2) for i in coordinatesDST]

In [7]:
# normalize information
scFeatures = MinMaxScaler()
distance_sensorData = scFeatures.fit_transform(distance_sensorData)
x_sensorData = scFeatures.fit_transform(x_sensorData)
y_sensorData = scFeatures.fit_transform(y_sensorData)
z_sensorData = scFeatures.fit_transform(z_sensorData)
depthSRC = scFeatures.fit_transform(depthSRC)
depthDST = scFeatures.fit_transform(depthDST)

In [8]:
df_target = pd.DataFrame({'x_displacement':target_dataX,'y_displacement':target_dataY})

In [9]:
displacement_depth = 3
if not featuresAreCoordinates:
    feature_data = []
    for i in range(dataSet_len):
        if depthStatus:
            displacement_depth = 4
            feature_data.append([x_sensorData[i], y_sensorData[i], distance_sensorData[i], z_sensorData[i]])
        else:
            feature_data.append([x_sensorData[i], y_sensorData[i], distance_sensorData[i]])
    feature_data = np.array(feature_data)
    print(feature_data.shape)
    # feature_data = feature_data.transpose((0, 2, 1))
    feature_data = feature_data.reshape(feature_data.shape[0], 1, -1)
    print(feature_data.shape)

In [10]:
coordinates_depth = 4
if featuresAreCoordinates:
    if depthStatus:
        coordinates_depth = 6
        depthSRCList = depthSRC.tolist()
        depthDSTList = depthDST.tolist()
        for i in range(dataSet_len):
            coordinatesSRC[i].extend(depthSRCList[i])
            coordinatesSRC[i].extend(coordinatesDST[i])
            coordinatesSRC[i].extend(depthDSTList[i])
    else:
        for i in range(dataSet_len):
            coordinatesSRC[i].extend(coordinatesDST[i])

    feature_data = np.array(coordinatesSRC)
    print(feature_data.shape)

(267, 306)


In [11]:
fig,ax = plt.subplots(1, figsize=(7,5))
# x_coordi = [i[0] for i in camera_displacement]
# ax.scatter(target_dataX, target_dataY)
x_coordi, y_coordi = [0], [0]
ctr = 0
for i, j in zip(target_dataX.to_list(), target_dataY.to_list()):
    x_coordi.append(i +  x_coordi[ctr])
    y_coordi.append(j +  y_coordi[ctr])
    ctr +=1
ax.plot(x_coordi, y_coordi)
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.show()

In [12]:
sc = MinMaxScaler()
target_data = sc.fit_transform(df_target)
plt.plot( target_data, label = 'target_data normalized')
plt.legend()
plt.show()

## Dataloading

In [13]:
def scaling_window(data, seq_length):
    x = []
    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        x.append(_x)

    return np.array(x)
if featuresAreCoordinates:
    feature_data = scaling_window(feature_data, sequence_len)
    target_data = target_data[:feature_data.shape[0]]
    dataSet_len = feature_data.shape[0]

In [14]:
learning_rate = 0.001
epochs = 1500
if featuresAreCoordinates:
    input_size = meanOfSensors*coordinates_depth
else:
    input_size = meanOfSensors*displacement_depth
hidden_size = 400
num_layers = 1 # use only one LSTM cell
output_size = 2
Shuff = True
if testModel:
    Shuff = False
X_train, X_test, y_train, y_test = train_test_split(feature_data, target_data, test_size = 0.3, shuffle= Shuff)

dataX = torch.Tensor(feature_data)
dataY = torch.Tensor(target_data)

trainX = torch.Tensor(X_train)
trainY = torch.Tensor(y_train)

testX = torch.Tensor(X_test)
testY = torch.Tensor(y_test)


in RNN, Weights are initialized in the recurrent neural network using normal distribution with zero mean. W1 will represent acceptance of input variables and w2 will represent the output which is generated as shown below.
## Model

In [15]:
class LSTM(nn.Module):

    def __init__(self, output_size, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        # define the LSTM layer
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        # define the output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Forward pass through the LSTM layer
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)
        
        return out

In [16]:
lstm = LSTM(output_size, input_size, hidden_size, num_layers)
if testModel:
    print("loading model ...")
    lstm.load_state_dict(torch.load('MODELS/KITTI/XYZ-all_SIFT.pth'))
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
train_lossList = []
test_lossList = []

loading model ...


In [17]:
if not testModel:
    lstm.train()
    # Train the model
    for epoch in range(epochs):
        # Clear stored gradient
        lstm.zero_grad()
        
        # forward pass
        outputs_train = lstm(trainX)
        # Zero out gradient, else they will accumulate between epoch
        optimizer.zero_grad()
        # obtain the loss function
        loss_train = criterion(outputs_train, trainY)
        if loss_train < 1:
            train_lossList.append(loss_train.item())

        outputs_test = lstm(testX)
        loss_test = criterion(outputs_test, testY)
        if loss_train < 1:
            test_lossList.append(loss_test.item())

        loss_train.backward() # computes the gradients
        optimizer.step() # updates the weights
        if epoch % 100 == 0:
            print("Epoch: %d, MSE loss: %1.5f" % (epoch, loss_train.item()))
        if epoch == (epochs-1):
            print("MSE:  %1.5f, RMSE: %1.5f" % (loss_train.item(), math.sqrt(loss_train.item())))

In [18]:
#Saving a checkpoint
if not testModel:
    print("saving model ...")
    torch.save(lstm.state_dict(), 'MODELS/KITTI/XYZ-all_SIFT.pth')

In [19]:
lstm.eval() # sets the model to evaluation mode
test_predict = lstm(testX)
real_loss = criterion(test_predict, testY)
test_predict = test_predict.data.numpy()

if not testModel:
    plotter = PredictionPlotter(title="Camera displacement prediction", save_to_filepath="PLOTS/KITTI/XYZ-all_SIFT_Prediction-TEST.png")

    # add them all
    for frame in range(len(testX)):
        # deactivate redrawing after each update
        plotter.add_values(frame, loss_train=testY[:,0][frame], loss_val=test_predict[:,0][frame], acc_train=testY[:,1][frame], acc_val=test_predict[:,1][frame], redraw=False)

    # redraw once at the end
    plotter.redraw()


plt.plot(test_predict[:,0], label = "real X")
plt.plot(testY[:,0], label = "prediction X",linestyle=":", alpha=0.8)
plt.plot(test_predict[:,1], label = "real Y")
plt.plot(testY[:,1], label = "prediction Y", linestyle=":", alpha=0.8)

plt.suptitle('Test Normalized Displacement Prediction')
plt.legend()
plt.show()

print("MSE:  %1.5f, RMSE: %1.5f" % (real_loss.item(), math.sqrt(real_loss.item())))


In [20]:
train_predict = lstm(dataX)
real_loss = criterion(train_predict, dataY)
data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

print(f"TOTAL RMSE: {math.sqrt(real_loss.item())}")

plt.plot(dataY_plot[:,0], label = "real X")
plt.plot(data_predict[:,0], label = "prediction X",linestyle=":", alpha=0.8)
plt.plot(dataY_plot[:,1], label = "real Y")
plt.plot(data_predict[:,1], label = "prediction Y", linestyle=":", alpha=0.8)

plt.suptitle('Local Normalized Displacement Prediction')
plt.legend()
plt.show()

In [21]:
dataSet_lenList = [i for i in range(dataSet_len)]
plt.scatter(dataSet_lenList, dataY_plot[:,1], s=1)
plt.scatter(dataSet_lenList, data_predict[:,1], s=1)
plt.suptitle('Local Normalized Scatter Displacement Prediction')
plt.show()

In [22]:
# create empty table with 12 fields
trainPredict_dataset_like = np.zeros(shape=(len(target_data), 2) )

# put the predicted values in the right field
trainPredict_dataset_like[:, 0] = data_predict[:,0]
trainPredict_dataset_like[:, 1] = data_predict[:,1]

# inverse transform and then select the right field
data_predictX = sc.inverse_transform(trainPredict_dataset_like)[:,0]
data_predictY = sc.inverse_transform(trainPredict_dataset_like)[:,1]

trainPredict_dataset_like[:, 0] = dataY_plot[:,0]
trainPredict_dataset_like[:, 1] = dataY_plot[:,1]
data_realX = sc.inverse_transform(trainPredict_dataset_like)[:,0]
data_realY = sc.inverse_transform(trainPredict_dataset_like)[:,1]

if testModel:
    pred_plotter = PredictionPlotter(title="Camera displacement prediction", save_to_filepath="PLOTS/KITTI/XYZ0095-all_SIFT_Prediction.png")
else:
    pred_plotter = PredictionPlotter(title="Camera displacement prediction", save_to_filepath="PLOTS/KITTI/XYZ-all_SIFT_Prediction.png")


# add them all
for frame in range(len(data_realX)):
    # deactivate redrawing after each update
    pred_plotter.add_values(frame, loss_train=data_realX[frame], loss_val=data_predictX[frame], acc_train=data_realY[frame], acc_val=data_predictY[frame], redraw=False)

# redraw once at the end
pred_plotter.redraw()

In [23]:
if testModel:
    import csv
    row_list = [["x_traget", "y_target", "x_prediction", "y_prediction"]]
    row_list.append([target_dataX.to_list(), target_dataY.to_list(), data_predictX.tolist(), data_predictY.tolist()])
    with open('KITTI_temp.csv', 'w', newline='') as myfile:
        writer = csv.writer(myfile, quoting=csv.QUOTE_NONNUMERIC, delimiter=',')
        writer.writerows(row_list)

In [24]:
fig,ax = plt.subplots(1, figsize=(7,5))
# x_coordi = [i[0] for i in camera_displacement]
# ax.scatter(target_dataX, target_dataY)
x_coordi_pred, y_coordi_pred = [0], [0]

ctr = 0
for i, j in zip(data_predictX, data_predictY):
    x_coordi_pred.append(i +  x_coordi_pred[ctr])
    y_coordi_pred.append(j +  y_coordi_pred[ctr])
    ctr +=1
ax.plot(x_coordi, y_coordi, label = "real")
ax.plot(x_coordi_pred, y_coordi_pred, label = "prediction")
ax.set_xlabel('x displacement (meters')
ax.set_ylabel('y displacement (meters)')
plt.suptitle('Accumulated Displacement Prediction')
plt.legend()
plt.show()

In [25]:
# generate some example values for the loss training line
if not testModel:
    plotter = LossAccPlotter(show_acc_plot=False, save_to_filepath="PLOTS/KITTI/XYZ-all_SIFT_Loss.png")
    train_lossList = np.array(train_lossList)
    test_lossList = np.array(test_lossList)

    # add them all
    for epoch, value in enumerate(train_lossList):
        # deactivate redrawing after each update
        plotter.add_values(epoch, loss_train=value, loss_val=test_lossList[epoch], redraw=False)

    # redraw once at the end
    plotter.redraw()

    plotter.block()